#### 타이타닉 데이터 분석 2단계
- Feature Engineering & 모델링
1. Feature Engineering은 모델의 성능을 향상 시키기 위해 데이터를 변환하고 새로운 특징을 생성하는 과정이다.
    - 기존 컬럼을 없애고 새로운 컬럼을 추가하기(과대적합을 방지할 목적으로 사용한다.)
2 주요 목적
    - 모델 성능 향상
    - 데이터 이해도 증진
    - 모델 학습 시간 단축 : 중요하지 않거나 중복된 특징은 제거한다.
3. 주요 기법
    - 데이터 변환
    - 인코딩 :  범주형 데이터를 수치형 데이터로 변환
        -  One-Hot-Encoding
    - 결측치 처리 : 데이터 일관성 유지 예) 평균값으로 채움, 중앙값으로 채우기, 삭제하기,  임의 예측값 채우기(허황된 생각)

- train과 test가 왜 나누어져 있는지?
- test데이터는 언제 쓰는 것인가?
- train과 test컬럼이 일치하는가?
- 보통 비율은 train: 80%, test: 20%
- 영향이 없는 컬럼은 제거하기 - drop 
- 상관 매트릭스  - df_train.corr() -> -1 ~ 1 사이의 값
- 모델들에 대한 손실율을 계산하는 방법

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm # 한글 폰트 관련
import matplotlib
from matplotlib import rc
path = "../data0225/malgun.ttf"
font_name = fm.FontProperties(fname=path).get_name()
matplotlib.rc("font", family = font_name)
plt.rc("font", family="Malgun Gothic")

In [2]:
import warnings
warnings.filterwarnings(action="ignore")

In [3]:
# 모형 - 훈련데이터
df_train = pd.read_csv("../data0225/titanic0225/train0225.csv")

# 답안지 - 테스트데이터
#테스트 데이터 - 답안지 - test.csv
df_test = pd.read_csv("../data0225/titanic0225/test0225.csv")

In [4]:
df_train.shape, df_test.shape

((891, 12), (418, 11))

In [5]:
df_train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [6]:
df_test.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [8]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
# drop메소드 사용해서 컬럼을 제거할 수 있다.
# inplace옵션에 True이면 원본이 바뀐다. - IO가 발생한다| 하지 않는다.
df_train.drop(columns=["Name", "Sex", "Cabin", "Embarked", "Ticket"], inplace=True)

In [10]:
df_train.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
0,1,0,3,22.0,1,0,7.2500
1,2,1,1,38.0,1,0,71.2833
2,3,1,3,26.0,0,0,7.9250
3,4,1,1,35.0,1,0,53.1000
4,5,0,3,35.0,0,0,8.0500


- corr()함수는 수치형 열들간의 상관계수를 계산하여 상관계수 행렬을 반환한다.
- 행렬에서 대각선 방향은 자기자신과의 상관관계이므로 1이다. - 배제함
- 대각선을 기준으로 위쪽이나 아래쪽 한쪽만 보면 된다.
    - 위치가 다른 것이지 위, 아래가 같은 값을 갖는다.
- 상관계수의 기본 개념
    - 1, 0, -1
    - 1 : 완벽한 양의 상관관계(두 변수가 함께 증가하거나 감소)
    - -1: 완벽한 음의 상관관계(한 변수가 증가 할 때 다른 변수가 감소)
    - 0 : 아무런 상관관계가 없다.

In [11]:
df_train.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,0.036847,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.057527,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


결측치를 확인해 보자

In [12]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Age            177
SibSp            0
Parch            0
Fare             0
dtype: int64

In [13]:
df_train = pd.read_csv("../data0225/titanic0225/train0225.csv")

In [14]:
# 0이면 여성 평균  - 28세
# 1이면 남성 평균 -  31세
sm = df_train.groupby("Sex")["Age"].mean()
sm

Sex
female    27.915709
male      30.726645
Name: Age, dtype: float64

In [15]:
type(df_train)

pandas.core.frame.DataFrame

In [16]:
# 판다스가 제공하는 DataFrame을 리스트로 변환한다.
data_list = [df_train]
type(data_list)

list

In [17]:
for data in data_list:
    data["Sex"] = data["Sex"].astype("category").cat.codes
df_train["Sex"].head(5)

0    1
1    0
2    0
3    0
4    1
Name: Sex, dtype: int8

In [18]:
# for문 사용하여 Age결측처리
for data in data_list:
    data.loc[(data["Sex"] == 0) & (data["Age"].isnull()),"Age"] = sm[0]
    data.loc[(data["Sex"] == 1) & (data["Age"].isnull()),"Age"] = sm[1]

In [19]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64